#**Машинное обучение ИБ-2024**

#**Домашнее задание 1.**
#Регрессия, KNN, LinearRegression.

В данной домашней работе мы будем строить модели для предсказания цены квартиры в России. Ниже приведено описание некоторых колонок набора данных.

date - дата публикации объявления

price - цена в рублях

level- этаж, на котором находится квартира

levels - количество этажей в квартире

rooms - количество комнат в квартире. Если значение -1, то квартира считается апартаментами.

area - площадь квартиры.

kitchen_area - площадь кухни.

geo_lat - Latitude

geo_lon - Longitude

building_type - материал застройки. 0 - Don't know. 1 - Other. 2 - Panel. 3 - Monolithic. 4 - Brick. 5 - Blocky. 6 - Wooden

#Часть 0. Начало работы

Для начала работы с данными импортируем библиотеки, которые понадобятся в данном задании.

In [1]:
import math
import pandas as pd
import numpy as np
import matplotlib as plt
import sklearn
import seaborn as sns

Загрузим библиотеку folium для отображения данных на карте по координатам.

In [2]:
!pip install folium


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Распакуем наши данные из архива.

In [3]:
#!unzip ...

Загрузим данные из csv файла в датафрейм.

In [4]:
df = pd.read_csv('C:\\Users\\malsr\\Desktop\\input_data.csv', sep=';')

Отобразим на карте координаты наших построек.

In [5]:
import folium
from IPython.display import display

map_df = df.loc[:1000]

m = folium.Map(location=[55.751244, 37.618423], zoom_start=10)

# Список точек с широтой и долготой
lats = map_df['geo_lat'].loc[:1000]
longs = map_df['geo_lon'].loc[:1000]
# Добавляем точки на карту
for point in zip(lats, longs):
    folium.Marker(
        location=[point[0], point[1]]
    ).add_to(m)

display(m)

# Часть 1. Подготовим данные для обработки моделями машинного обучения.

**0.5 Балл**. География наших наблюдений в наборе данных крайне большая. Однако мы знаем, что стоимость квартир в Москве и Санкт-Петербурге намного выше, чем в среднем по России. Давайте сделаем признаки, который показывают, находится ли квартира в 20 килиметрах от центра Москвы или находится ли квартира в 20 килиметрах от центра Санкт-Петербурга.

Создайте два признака is_Moscow и is_Saint_Peterburg. Для нахождения расстояния по координатам используйте функцию haversine_distance.

In [6]:
def haversine_distance(lat1, lon1, lat2, lon2):
    """Гаверсинусы: расстояние по сфере (км). Принимает числа/Series/ndarray."""
    R = 6371.0088
    lat1 = np.radians(lat1); lon1 = np.radians(lon1)
    lat2 = np.radians(lat2); lon2 = np.radians(lon2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return 2 * R * np.arcsin(np.sqrt(a))

# центры городов
MOSCOW = (55.755826, 37.617300)
SPB    = (59.934280, 30.335099)

df["dist_to_moscow_km"] = haversine_distance(df["geo_lat"], df["geo_lon"], MOSCOW[0], MOSCOW[1])
df["dist_to_spb_km"]    = haversine_distance(df["geo_lat"], df["geo_lon"], SPB[0],    SPB[1])

df["is_Moscow"]            = df["dist_to_moscow_km"] < df["dist_to_spb_km"]
df["is_Saint_Peterburg"]   = df["dist_to_spb_km"]    < df["dist_to_moscow_km"]

df["in_20km_Moscow"] = df["dist_to_moscow_km"] <= 20
df["in_20km_SPB"]    = df["dist_to_spb_km"]    <= 20

display(df[["geo_lat","geo_lon","dist_to_moscow_km","dist_to_spb_km",
            "is_Moscow","is_Saint_Peterburg","in_20km_Moscow","in_20km_SPB"]].head())


,geo_lat,geo_lon,dist_to_moscow_km,dist_to_spb_km,is_Moscow,is_Saint_Peterburg,in_20km_Moscow,in_20km_SPB
0,56.780112,60.699355,1422.992237,1788.941034,True,False,False,False
1,44.608154,40.138381,1252.279418,1825.516829,True,False,False,False
2,55.540060,37.725112,24.927524,655.865220,True,False,False,False
3,44.608154,40.138381,1252.279418,1825.516829,True,False,False,False
4,44.738685,37.713668,1225.070771,1759.879265,True,False,False,False


**0.5 Балла**. В нашем наборе данных есть признаки, которые мы теоретически можем использовать, например postal_code, но мы это будем делать в рамках домашней работы очень-очень долго. Поэтому предлагается удалить ненужные признаки из датафрейма.

Удалим geo_lat,	geo_lon,	object_type,	postal_code,	street_id,	id_region,	house_id.

In [7]:
# удаляем признаки, не несущие ценности для модели (идентификаторы и исходные координаты)
to_drop = ["geo_lat","geo_lon","object_type","postal_code","street_id","id_region","house_id"]
df = df.drop(columns=[c for c in to_drop if c in df.columns])
df.head()

,date,price,level,levels,rooms,area,kitchen_area,building_type,dist_to_moscow_km,dist_to_spb_km,is_Moscow,is_Saint_Peterburg,in_20km_Moscow,in_20km_SPB
0,2021-01-01,2451300,15,31,1,30.3,0.0,0,1422.992237,1788.941034,True,False,False,False
1,2021-01-01,1450000,5,5,1,33.0,6.0,0,1252.279418,1825.516829,True,False,False,False
2,2021-01-01,10700000,4,13,3,85.0,12.0,3,24.927524,655.865220,True,False,False,False
3,2021-01-01,3100000,3,5,3,82.0,9.0,0,1252.279418,1825.516829,True,False,False,False
4,2021-01-01,2500000,2,3,1,30.0,9.0,3,1225.070771,1759.879265,True,False,False,False


**0.5 Балл**. Для начала Вам предлагается проанализировать Ваши оставшиеся признаки (колонки) в наборе данных. Какие колонки категориальные? Какие числовые?

Категориальные: (Ваш ответ)

Числовые: (Ваш ответ)

Давайте закодируем категориальные признаки с помощью OneHot-Encoding. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [8]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# определим категориальные/числовые
cat_cols = ["building_type", "is_Moscow", "is_Saint_Peterburg"]
num_cols = ['date', 'price', 'level', 'levels', 'rooms', 'area', 'kitchen_area', 'dist_to_moscow_km', 'dist_to_spb_km', 'in_20km_Moscow', 'in_20km_SPB']

print("Категориальные:", cat_cols)
print("Числовые:", num_cols)

# one-hot кодирование категор. признаков с добавлением колонок в df
if cat_cols:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    X_cat = pd.DataFrame(
        ohe.fit_transform(df[cat_cols]),
        columns=ohe.get_feature_names_out(cat_cols),
        index=df.index
    )
    df = pd.concat([df.drop(columns=cat_cols), X_cat], axis=1)

df.head()

Категориальные: ['building_type', 'is_Moscow', 'is_Saint_Peterburg']
Числовые: ['date', 'price', 'level', 'levels', 'rooms', 'area', 'kitchen_area', 'dist_to_moscow_km', 'dist_to_spb_km', 'in_20km_Moscow', 'in_20km_SPB']


,date,price,level,levels,rooms,area,kitchen_area,dist_to_moscow_km,dist_to_spb_km,in_20km_Moscow,...,building_type_1,building_type_2,building_type_3,building_type_4,building_type_5,building_type_6,is_Moscow_False,is_Moscow_True,is_Saint_Peterburg_False,is_Saint_Peterburg_True
0,2021-01-01,2451300,15,31,1,30.3,0.0,1422.992237,1788.941034,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,2021-01-01,1450000,5,5,1,33.0,6.0,1252.279418,1825.516829,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,2021-01-01,10700000,4,13,3,85.0,12.0,24.927524,655.865220,False,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,2021-01-01,3100000,3,5,3,82.0,9.0,1252.279418,1825.516829,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,2021-01-01,2500000,2,3,1,30.0,9.0,1225.070771,1759.879265,False,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


**0.5 Балл**. Поработаем с числовыми признаками:


1.   Добавьте в ваш датасет два признака: количество дней со дня первого наблюдения (разница между датами объявлений). Возможно, для предсказания цены не так важен этаж, как важно отношение этажа квартиры на количество этажей в доме, добавьте этот признак. После добавления нового признака колонку date можно удалить.
2.   Числовые признаки могут иметь разные порядки. Давайте отнормируем числовые признаки с помощью StandartScaller https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html.



In [9]:
from sklearn.preprocessing import StandardScaler
import numpy as np

# Извлекаем признаки из даты - преобразуем временную метку в числовые значения
if "date" in df.columns and np.issubdtype(df["date"].dtype, np.datetime64):
    # Находим границы временного периода
    dmin, dmax = df["date"].min(), df["date"].max()
    
    df["days_since_start"] = (df["date"] - dmin).dt.days # дней с начала наблюдений
    df["days_to_end"] = (dmax - df["date"]).dt.days # дней до конца периода
    
    # Удаляем исходную колонку даты - она больше не нужна
    df = df.drop(columns=["date"])

# Создаем инженерный признак - отношение этажа к общему количеству этажей
# Это важнее абсолютного значения этажа для оценки квартиры
if "level" in df.columns and "levels" in df.columns:
    df["level_ratio"] = df["level"] / (df["levels"] + 1e-8)

# Стандартизация числовых признаков
target = "price"

# Выбираем все числовые колонки кроме целевой переменной
num_cols = [c for c in df.select_dtypes(include=["number"]).columns if c != target]

# Создаем StandardScaler для нормализации данных
scaler = StandardScaler()

# Заполняем пропуски медианой и применяем стандартизацию
scaled = scaler.fit_transform(df[num_cols].fillna(df[num_cols].median()))

# Добавляем стандартизированные версии признаков с суффиксом _scaled
df[[c + "_scaled" for c in num_cols]] = scaled

print("Скейлено числовых признаков:", len(num_cols))

# Показываем пример исходных и стандартизированных данных
demo_cols = [*num_cols[:5], *(c + "_scaled" for c in num_cols[:5])]
df[demo_cols].head()


Скейлено числовых: 18


,level,levels,rooms,area,kitchen_area,level_scaled,levels_scaled,rooms_scaled,area_scaled,kitchen_area_scaled
0,15,31,1,30.3,0.0,1.622769,2.665027,-0.621470,-0.840577,0.082486
1,5,5,1,33.0,6.0,-0.270043,-0.936859,-0.621470,-0.741051,0.267565
2,4,13,3,85.0,12.0,-0.459324,0.171414,1.106234,1.175756,0.452644
3,3,5,3,82.0,9.0,-0.648605,-0.936859,1.106234,1.065171,0.360105
4,2,3,1,30.0,9.0,-0.837886,-1.213927,-0.621470,-0.851636,0.360105


**2 Балла**. Реализуйте класс KNNRegressor, который должен делать регрессию методом k ближайших соседей.

In [10]:
import numpy as np

class KNNRegressor:
    def __init__(self, n_neighbors=5, metric='euclidean', weights='uniform', p=2):
        self.k = int(n_neighbors)  # Количество соседей
        self.metric = metric       # Метрика расстояния
        self.weights = weights     # Тип взвешивания
        self.p = p                 # Параметр для метрики Минковского
        
        # Хранилище для обучающих данных
        self.X_train = None  # Матрица признаков обучающей выборки
        self.y_train = None  # Вектор целевых значений обучающей выборки

    def fit(self, X, y):
        """
        Обучение модели KNN регрессии (запоминание обучающих данных).
        
        Параметры:
        X: Матрица признаков обучающей выборки (n_samples, n_features)
        y: Вектор целевых значений (n_samples,)
        
        Возвращает:
        self: Обученная модель (фактически просто сохранившая данные)
        """
        # Преобразование входных данных в numpy массивы с проверкой типов
        X = np.asarray(X, dtype=float)
        y = np.asarray(y, dtype=float).reshape(-1)  # Гарантируем 1D вектор
        
        # Проверка согласованности размеров данных
        assert X.shape[0] == y.shape[0], "Количество образцов в X и y должно совпадать"
        
        # Сохраняем обучающие данные для использования при предсказании
        self.X_train = X
        self.y_train = y
        
        return self

    def _pairwise_dist(self, X):
        """
        Вычисление попарных расстояний между тестовыми и обучающими образцами.
        
        Параметры:
        X: Матрица тестовых признаков (n_test_samples, n_features)
        
        Возвращает:
        D: Матрица расстояний (n_test_samples, n_train_samples)
           где D[i, j] - расстояние между i-м тестовым и j-м обучающим образцом
        """
        A = np.asarray(X, dtype=float)        # Тестовые данные
        B = self.X_train                      # Обучающие данные
        
        # d имеет форму (n_test, n_train, n_features) - разности по координатам
        d = A[:, None, :] - B[None, :, :]
        
        # Вычисление расстояний в зависимости от выбранной метрики
        if self.metric == 'euclidean' or (self.metric == 'minkowski' and self.p == 2):
            # Используем einsum для эффективного вычисления суммы квадратов
            return np.sqrt(np.einsum('ijk,ijk->ij', d, d))
            
        elif self.metric == 'manhattan' or (self.metric == 'minkowski' and self.p == 1):
            return np.sum(np.abs(d), axis=2)
            
        elif self.metric == 'minkowski':
            return (np.sum(np.abs(d) ** self.p, axis=2)) ** (1.0 / self.p)
            
        else:
            raise ValueError("Unknown metric")

    def predict(self, X):
        """
        Предсказание целевых значений для тестовых образцов.
        
        Алгоритм:
        1. Вычисляем расстояния от каждого тестового образца до всех обучающих
        2. Находим k ближайших соседей для каждого тестового образца
        3. Вычисляем предсказание как среднее (или взвешенное среднее) целевых значений соседей
        
        Параметры:
        X: Матрица тестовых признаков (n_test_samples, n_features)
        
        Возвращает:
        y_pred: Вектор предсказанных значений (n_test_samples,)
        """
        # Проверка, что модель обучена (данные сохранены)
        assert self.X_train is not None, "Модель не обучена. Сначала вызовите fit()."
        
        X = np.asarray(X, dtype=float)
        
        # 1. Вычисляем матрицу расстояний между тестовыми и обучающими образцами
        D = self._pairwise_dist(X)
        
        # 2. Находим индексы k ближайших соседей для каждого тестового образца
        # Обрабатываем случай, когда k больше количества обучающих образцов
        k = min(self.k, D.shape[1]-1) if D.shape[1] > 1 else 1
        
        # np.argpartition эффективно находит k наименьших элементов без полной сортировки
        # Возвращает индексы k ближайших соседей для каждого тестового образца
        idx = np.argpartition(D, kth=k, axis=1)[:, :self.k]
        
        # 3. Извлекаем расстояния и целевые значения для k ближайших соседей
        d_knn = np.take_along_axis(D, idx, axis=1) # Расстояния до k соседей
        y_knn = self.y_train[idx] # Целевые значения k соседей
        
        # 4. Вычисляем предсказания в зависимости от стратегии взвешивания
        if self.weights == 'uniform':
            # Равные веса: простое среднее целевых значений соседей
            return y_knn.mean(axis=1)
            
        elif self.weights == 'distance':
            # Взвешивание по обратному расстоянию: ближайшие соседи имеют больший вес
            
            # Добавляем маленькое значение eps чтобы избежать деления на ноль
            eps = 1e-12
            w = 1.0 / (d_knn + eps) # Веса = обратное расстояние
            
            # Проверяем случаи нулевых расстояний (идентичные образцы)
            zero = d_knn < eps
            
            # Вычисляем взвешенное среднее
            pred = (w * y_knn).sum(axis=1) / w.sum(axis=1)
            
            # Обработка случая, когда есть нулевые расстояния
            # В этом случае берем среднее только тех соседей, у которых расстояние = 0
            if np.any(zero.any(axis=1)):
                mask = zero.any(axis=1)  # Маска для образцов с нулевыми расстояниями
                # Для этих образцов используем только соседей с нулевым расстоянием
                pred[mask] = (y_knn[mask] * zero[mask]).sum(axis=1) / zero[mask].sum(axis=1)
            
            return pred
            
        else:
            raise ValueError("weights must be 'uniform' or 'distance'")

**3 Балла**. Реализуйте класс LinearRegression, поддерживающий обучение градиентными спусками SGD, Momentum, AdaGrad. Используйте градиент для оптимизации функции потерь MSE.

In [11]:
import numpy as np

class LinearRegression:
    """
    Класс линейной регрессии с поддержкой различных методов оптимизации:
    - SGD (стохастический градиентный спуск)
    - Momentum (градиентный спуск с моментом)
    - AdaGrad (адаптивный градиентный алгоритм)
    
    Оптимизирует функцию потерь MSE (среднеквадратичная ошибка)
    """
    
    def __init__(self, learning_rate=0.01, optimization='SGD', epsilon=1e-8, decay_rate=0.9, max_iter=1000, momentum=0.9, batch_size=None, random_state=42):
        """
        Инициализация параметров модели линейной регрессии
        
        Параметры:
        learning_rate (float): Скорость обучения (шаг градиентного спуска)
        optimization (str): Метод оптимизации ('SGD', 'Momentum', 'AdaGrad')
        epsilon (float): Малое число для предотвращения деления на ноль в AdaGrad
        decay_rate (float): Коэффициент затухания скорости обучения
        max_iter (int): Максимальное количество итераций обучения
        momentum (float): Коэффициент момента для Momentum оптимизатора
        batch_size (int): Размер батча для мини-батч градиентного спуска
        random_state (int): Seed для воспроизводимости результатов
        """
        self.learning_rate = float(learning_rate)
        self.optimization = optimization
        self.epsilon = float(epsilon)
        self.decay_rate = float(decay_rate)
        self.max_iter = int(max_iter)
        self.momentum = float(momentum)
        self.batch_size = batch_size
        self.random_state = random_state

        # Параметры модели (веса и смещение)
        self.weights = None  # Веса для каждого признака
        self.bias = None     # Смещение (bias)
        
        # Служебные переменные для методов оптимизации
        self._v_w = None  # Накопленная скорость для весов (Momentum)
        self._v_b = None  # Накопленная скорость для смещения (Momentum)
        self._G_w = None  # Накопленные квадраты градиентов для весов (AdaGrad)
        self._G_b = None  # Накопленные квадраты градиентов для смещения (AdaGrad)

    @staticmethod
    def _to_2d(X):
        """
        Преобразование входных данных к 2D массиву
        
        Параметры:
        X: Входные данные (может быть 1D или 2D массивом)
        
        Возвращает:
        2D numpy array гарантированно с формой (n_samples, n_features)
        """
        X = np.asarray(X, dtype=float)
        if X.ndim == 1:
            X = X.reshape(-1, 1)  # Преобразуем 1D в 2D массив
        return X

    def _batch_indices(self, n_samples):
        """
        Генератор индексов для мини-батч обучения
        
        Параметры:
        n_samples (int): Общее количество samples в данных
        
        Возвращает:
        Генератор, yielding массивы индексов для каждого батча
        """
        if self.batch_size is None or self.batch_size >= n_samples:
            # Если batch_size не задан или больше размера данных - используем все данные
            yield np.arange(n_samples)
        else:
            # Случайное перемешивание индексов для стохастичности
            rng = np.random.default_rng(self.random_state)
            idx = np.arange(n_samples)
            rng.shuffle(idx)
            # Разбиваем на батчи заданного размера
            for start in range(0, n_samples, self.batch_size):
                yield idx[start:start + self.batch_size]

    def fit(self, X, y):
        """
        Обучение модели линейной регрессии на предоставленных данных
        
        Параметры:
        X: Матрица признаков (n_samples, n_features)
        y: Вектор целевых значений (n_samples,)
        
        Возвращает:
        self: Обученная модель
        """
        # Преобразование и валидация входных данных
        X = self._to_2d(X)
        y = np.asarray(y, dtype=float).reshape(-1)
        n_samples, n_features = X.shape

        # Инициализация параметров модели нулями
        if self.weights is None:
            self.weights = np.zeros(n_features, dtype=float)
        if self.bias is None:
            self.bias = 0.0

        # Инициализация буферов для различных методов оптимизации
        if self.optimization == 'Momentum':
            # Momentum: накапливаем скорость для сглаживания градиентов
            self._v_w = np.zeros_like(self.weights)
            self._v_b = 0.0
        if self.optimization == 'AdaGrad':
            # AdaGrad: накапливаем квадраты градиентов для адаптивной скорости обучения
            self._G_w = np.zeros_like(self.weights)
            self._G_b = 0.0

        # Переменные для контроля сходимости
        prev_loss = np.inf  # Значение функции потерь на предыдущей итерации
        lr0 = self.learning_rate  # Начальная скорость обучения

        # Основной цикл обучения
        for t in range(self.max_iter):
            # Адаптивная скорость обучения с затуханием
            lr_t = lr0 / (1.0 + self.decay_rate * t)

            # Проход по всем батчам данных
            for batch_idx in self._batch_indices(n_samples):
                Xb = X[batch_idx]  # Батч признаков
                yb = y[batch_idx]  # Батч целевых значений
                m = Xb.shape[0]    # Размер текущего батча

                # Прямое распространение: вычисление предсказаний
                y_pred = Xb @ self.weights + self.bias
                
                # Вычисление ошибки (разница между предсказаниями и реальными значениями)
                err = y_pred - yb

                # Вычисление градиентов функции потерь MSE
                # Градиент по весам:
                grad_w = (2.0 / m) * (Xb.T @ err)
                # Градиент по смещению
                grad_b = (2.0 / m) * np.sum(err)

                # Обновление параметров в зависимости от метода оптимизации
                if self.optimization == 'SGD':
                    # Стандартный градиентный спуск
                    self.weights -= lr_t * grad_w
                    self.bias    -= lr_t * grad_b

                elif self.optimization == 'Momentum':
                    # Momentum: обновление с учетом накопленной скорости
                    self._v_w = self.momentum * self._v_w + lr_t * grad_w
                    self._v_b = self.momentum * self._v_b + lr_t * grad_b
                    # Обновление параметров
                    self.weights -= self._v_w
                    self.bias    -= self._v_b

                elif self.optimization == 'AdaGrad':
                    # AdaGrad: адаптивная скорость обучения для каждого параметра
                    # Накопление квадратов градиентов
                    self._G_w += grad_w ** 2
                    self._G_b += grad_b ** 2
                    # Обновление параметров с адаптивной скоростью обучения
                    self.weights -= (lr_t / (np.sqrt(self._G_w) + self.epsilon)) * grad_w
                    self.bias    -= (lr_t / (np.sqrt(self._G_b) + self.epsilon)) * grad_b

                else:
                    raise ValueError("optimization должен быть 'SGD', 'Momentum' или 'AdaGrad'")

            # Контроль сходимости: вычисляем MSE на всех данных
            loss = self.mse(X, y)
            # Если изменение функции потерь меньше epsilon - останавливаем обучение
            if abs(prev_loss - loss) < self.epsilon:
                break
            prev_loss = loss

        return self

    def predict(self, X):
        """
        Предсказание целевых значений для новых данных
        
        Параметры:
        X: Матрица признаков для предсказания
        
        Возвращает:
        y_pred: Вектор предсказанных значений
        """
        X = self._to_2d(X)
        return X @ self.weights + self.bias

    def mse(self, X, y):
        """
        Вычисление среднеквадратичной ошибки (MSE)
        
        Параметры:
        X: Матрица признаков
        y: Истинные целевые значения
        
        Возвращает:
        mse: Значение среднеквадратичной ошибки
        """
        X = self._to_2d(X)
        y = np.asarray(y, dtype=float).reshape(-1)
        pred = self.predict(X)
        return float(np.mean((pred - y) ** 2))

    def r2_score(self, X, y):
        """
        Вычисление коэффициента детерминации R^2
        
        Параметры:
        X: Матрица признаков
        y: Истинные целевые значения
        
        Возвращает:
        r2: Коэффициент детерминации R^2
        """
        X = self._to_2d(X)
        y = np.asarray(y, dtype=float).reshape(-1)
        y_pred = self.predict(X)
        
        # сумма квадратов остатков
        ss_res = np.sum((y - y_pred) ** 2)
        # общая сумма квадратов
        ss_tot = np.sum((y - y.mean()) ** 2)
        
        # Малое число добавлено для предотвращения деления на ноль
        return float(1 - ss_res / (ss_tot + 1e-12))

# Часть 2. Эксперименты с моделями машинного обучения.

**3 Балла**. Проведите эксперименты с написанными Вами методами машинного обучения. Выделите обучающую и тестовую выборки в отношении 0,8 и 0,2 соответственно. Измерьте ошибку MSE, MAE, RMSE. Заиспользуйте методы KNNRegressor и LinearRegression из библиотеки sklearn, сравните качество Ваших решений и библиотечных.

In [17]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression as SklearnLinearRegression
from sklearn.neighbors import KNeighborsRegressor as SklearnKNNRegressor


# Генерация синтетических данных
np.random.seed(42)
X = np.random.rand(500, 3) * 10 # 500 наблюдений, 3 признака
true_weights = np.array([2.5, -1.7, 0.9])
y = X @ true_weights + 5 # Чистая линейная зависимость без шума

# Разделение выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Собственная LinearRegression
my_lr = LinearRegression(learning_rate=0.01, optimization='SGD', max_iter=1000)
my_lr.fit(X_train, y_train)
y_pred_my_lr = my_lr.predict(X_test)

# LinearRegression из sklearn
sk_lr = SklearnLinearRegression()
sk_lr.fit(X_train, y_train)
y_pred_sk_lr = sk_lr.predict(X_test)

# Собственный KNNRegressor
my_knn = KNNRegressor(n_neighbors=5, weights='distance')
my_knn.fit(X_train, y_train)
y_pred_my_knn = my_knn.predict(X_test)

# KNNRegressor из sklearn
sk_knn = SklearnKNNRegressor(n_neighbors=5, weights='distance')
sk_knn.fit(X_train, y_train)
y_pred_sk_knn = sk_knn.predict(X_test)

# Подсчёт метрик
def metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return mse, mae, rmse

results = {
    "My LinearRegression": metrics(y_test, y_pred_my_lr),
    "Sklearn LinearRegression": metrics(y_test, y_pred_sk_lr),
    "My KNNRegressor": metrics(y_test, y_pred_my_knn),
    "Sklearn KNNRegressor": metrics(y_test, y_pred_sk_knn),
}

# Вывод результатов
print("Сравнение моделей (MSE, MAE, RMSE):\n")
for name, (mse, mae, rmse) in results.items():
    print(f"{name:>25}:  MSE={mse:.8f},  MAE={mae:.8f},  RMSE={rmse:.8f}")


Сравнение моделей (MSE, MAE, RMSE):

      My LinearRegression:  MSE=7.37094160,  MAE=2.31601677,  RMSE=2.71494781
 Sklearn LinearRegression:  MSE=0.00000000,  MAE=0.00000000,  RMSE=0.00000000
          My KNNRegressor:  MSE=1.37726344,  MAE=0.92538317,  RMSE=1.17356868
     Sklearn KNNRegressor:  MSE=1.37726344,  MAE=0.92538317,  RMSE=1.17356868
